In [4]:
import os

In [2]:
# import sys
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [10]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification\\research'

In [11]:
os.chdir("../") # type: ignore

In [12]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
# ! pip install python-box
# ! pip install pyYAML
# ! pip install ensure==1.0.2


In [15]:
import sys
sys.path.append('/absolute/or/relative/path/to/src')

In [16]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.paramns = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import zipfile
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info: \n{headers}')
        else:
            logger.info(f'file already exists of size: {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-06-20 22:47:12,533:  INFO:  common:  yaml file: config\config.yaml loaded successfully]


[2025-06-20 22:47:12,693:  INFO:  common:  yaml file: params.yaml loaded successfully]
[2025-06-20 22:47:12,757:  INFO:  common:  created directory at: artifacts]
[2025-06-20 22:47:12,792:  INFO:  common:  created directory at: artifacts/data_ingestion]
[2025-06-20 22:48:42,484:  INFO:  3167276667:  artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DDCB:2AAF15:3A08E:8F1F3:685597EF
Accept-Ranges: bytes
Date: Fri, 20 Jun 2025 17:18:41 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100031-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1750439921.573103,VS0,VE906
Vary: Authorizatio

In [ ]:
# import yaml
# with open("config/config.yaml") as f:
#     content = yaml.safe_load(f)
# print(content)